In [2]:
# AI-POWERED & AUGMENTED REALITY FACE DETECTION
# Advanced features to make your project stand out

# ==================================================
# SECTION 1: AI-POWERED UPGRADES
# ==================================================

# Required installations:
# pip install mediapipe tensorflow opencv-contrib-python dlib

import cv2
import numpy as np
import mediapipe as mp
import time
import math

# ---------------------------------------------------
# 1.1 MODERN FACE DETECTION WITH MEDIAPIPE
# ---------------------------------------------------

class AIFaceDetector:
    def __init__(self):
        # Initialize MediaPipe Face Detection (more accurate than Haar cascades)
        self.mp_face_detection = mp.solutions.face_detection
        self.mp_drawing = mp.solutions.drawing_utils
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=0,  # 0 for close-range, 1 for full-range
            min_detection_confidence=0.5
        )
        
        # Initialize MediaPipe Face Mesh for detailed landmarks
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=5,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

    def detect_faces_ai(self, frame):
        """Advanced AI-powered face detection"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(rgb_frame)
        
        faces = []
        if results.detections:
            for detection in results.detections:
                # Get bounding box
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                
                x = int(bbox.xmin * w)
                y = int(bbox.ymin * h)
                width = int(bbox.width * w)
                height = int(bbox.height * h)
                
                # Get confidence score
                confidence = detection.score[0]
                
                faces.append({
                    'bbox': (x, y, width, height),
                    'confidence': confidence
                })
                
                # Draw detection
                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
                cv2.putText(frame, f'Confidence: {confidence:.2f}', 
                            (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        return frame, faces

    def detect_facial_landmarks(self, frame):
        """Detect 468 facial landmarks for precise tracking"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Draw face mesh
                self.mp_drawing.draw_landmarks(
                    frame, face_landmarks, self.mp_face_mesh.FACEMESH_CONTOURS,
                    None, self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
                )
                
                # Extract key points
                h, w, _ = frame.shape
                landmarks = []
                for landmark in face_landmarks.landmark:
                    x = int(landmark.x * w)
                    y = int(landmark.y * h)
                    landmarks.append((x, y))
                
                # Highlight key facial features
                self.highlight_facial_features(frame, landmarks)
        
        return frame

    def highlight_facial_features(self, frame, landmarks):
        """Highlight eyes, nose, and mouth"""
        if len(landmarks) >= 468:
            # Eye regions (approximate indices)
            left_eye = landmarks[33]  # Left eye center
            right_eye = landmarks[362]  # Right eye center
            nose_tip = landmarks[1]   # Nose tip
            mouth_center = landmarks[13]  # Mouth center
            
            # Draw feature points
            features = [
                (left_eye, "L_Eye", (255, 0, 0)),
                (right_eye, "R_Eye", (255, 0, 0)),
                (nose_tip, "Nose", (0, 255, 255)),
                (mouth_center, "Mouth", (0, 0, 255))
            ]
            
            for point, label, color in features:
                cv2.circle(frame, point, 5, color, -1)
                cv2.putText(frame, label, (point[0]+10, point[1]), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

# ---------------------------------------------------
# 1.2 AGE AND GENDER DETECTION
# ---------------------------------------------------

class AgeGenderDetector:
    def __init__(self):
        # Load pre-trained models (you need to download these)
        self.age_net = None
        self.gender_net = None
        self.load_models()
    
    def load_models(self):
        """Load age and gender detection models"""
        try:
            # Age detection model
            age_weights = "age_net.caffemodel"  # Download from OpenCV model zoo
            age_config = "age_deploy.prototxt"
            self.age_net = cv2.dnn.readNet(age_weights, age_config)
            
            # Gender detection model
            gender_weights = "gender_net.caffemodel"
            gender_config = "gender_deploy.prototxt"
            self.gender_net = cv2.dnn.readNet(gender_weights, gender_config)
            
            print("✅ Age and Gender models loaded successfully!")
        except:
            print("⚠️ Age/Gender models not found. Download from OpenCV model zoo.")
    
    def predict_age_gender(self, face_roi):
        """Predict age and gender for a face region"""
        if self.age_net is None or self.gender_net is None:
            return "Unknown", "Unknown"
        
        # Preprocess face
        blob = cv2.dnn.blobFromImage(face_roi, 1.0, (227, 227), 
                                    (78.4263377603, 87.7689143744, 114.895847746))
        
        # Age prediction
        self.age_net.setInput(blob)
        age_predictions = self.age_net.forward()
        age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
        predicted_age = age_list[age_predictions[0].argmax()]
        
        # Gender prediction
        self.gender_net.setInput(blob)
        gender_predictions = self.gender_net.forward()
        gender_list = ['Male', 'Female']
        predicted_gender = gender_list[gender_predictions[0].argmax()]
        
        return predicted_age, predicted_gender

# ==================================================
# SECTION 2: AUGMENTED REALITY FEATURES
# ==================================================

class ARFaceFilters:
    def __init__(self):
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5
        )
        
        # Load filter assets (you can create these or download)
        self.load_filter_assets()
    
    def load_filter_assets(self):
        """Load virtual mask and filter images"""
        try:
            # Create simple virtual elements
            self.sunglasses = self.create_sunglasses()
            self.mustache = self.create_mustache()
            self.hat = self.create_hat()
            print("✅ AR Filter assets loaded!")
        except Exception as e:
            print(f"⚠️ Filter assets error: {e}")
    
    def create_sunglasses(self):
        """Create a simple sunglasses overlay"""
        # Create a simple black rectangle for sunglasses
        sunglasses = np.zeros((40, 120, 4), dtype=np.uint8)  # RGBA
        sunglasses[:, :, :3] = [0, 0, 0]  # Black
        sunglasses[:, :, 3] = 200  # Semi-transparent
        
        # Add white frame
        cv2.rectangle(sunglasses, (0, 0), (119, 39), (255, 255, 255, 255), 2)
        cv2.line(sunglasses, (60, 20), (60, 20), (255, 255, 255, 255), 3)  # Bridge
        
        return sunglasses
    
    def create_mustache(self):
        """Create a simple mustache overlay"""
        mustache = np.zeros((30, 80, 4), dtype=np.uint8)
        # Draw mustache shape
        cv2.ellipse(mustache, (40, 15), (35, 10), 0, 0, 180, (101, 67, 33, 255), -1)
        return mustache
    
    def create_hat(self):
        """Create a simple hat overlay"""
        hat = np.zeros((60, 100, 4), dtype=np.uint8)
        cv2.rectangle(hat, (10, 40), (90, 55), (255, 0, 0, 255), -1)  # Hat brim
        cv2.rectangle(hat, (25, 10), (75, 45), (0, 0, 255, 255), -1)   # Hat top
        return hat
    
    def apply_sunglasses_filter(self, frame, landmarks):
        """Apply sunglasses to detected face"""
        if len(landmarks) >= 468:
            # Get eye positions
            left_eye = landmarks[33]
            right_eye = landmarks[362]
            
            # Calculate sunglasses position and size
            eye_distance = int(math.sqrt((right_eye[0] - left_eye[0])**2 + 
                                        (right_eye[1] - left_eye[1])**2))
            
            center_x = (left_eye[0] + right_eye[0]) // 2
            center_y = (left_eye[1] + right_eye[1]) // 2
            
            # Resize sunglasses based on face size
            scale = eye_distance / 60  # Adjust scale factor
            new_width = int(self.sunglasses.shape[1] * scale)
            new_height = int(self.sunglasses.shape[0] * scale)
            
            if new_width > 0 and new_height > 0:
                resized_glasses = cv2.resize(self.sunglasses, (new_width, new_height))
                
                # Position sunglasses
                start_x = center_x - new_width // 2
                start_y = center_y - new_height // 2
                
                # Apply overlay
                self.overlay_image(frame, resized_glasses, start_x, start_y)
    
    def overlay_image(self, background, overlay, x, y):
        """Overlay image with alpha channel onto background"""
        if overlay.shape[2] == 4:  # RGBA
            h, w = overlay.shape[:2]
            
            # Ensure overlay fits within frame
            if x + w <= background.shape[1] and y + h <= background.shape[0] and x >= 0 and y >= 0:
                # Extract alpha channel
                alpha = overlay[:, :, 3] / 255.0
                
                # Apply overlay
                for c in range(3):  # RGB channels
                    background[y:y+h, x:x+w, c] = (
                        alpha * overlay[:, :, c] + 
                        (1 - alpha) * background[y:y+h, x:x+w, c]
                    )

# ==================================================
# SECTION 3: COMBINED DEMO APPLICATION
# ==================================================

class AdvancedFaceApp:
    def __init__(self):
        self.ai_detector = AIFaceDetector()
        self.ar_filters = ARFaceFilters()
        self.age_gender = AgeGenderDetector()
        
        self.current_filter = "none"  # none, sunglasses, mustache, hat
        self.show_landmarks = False
        self.show_age_gender = False
    
    def run_advanced_detection(self):
        """Run the complete advanced face detection application"""
        cap = cv2.VideoCapture(0)
        
        print("🚀 Advanced AI Face Detection Started!")
        print("Controls:")
        print("  's' - Toggle sunglasses filter")
        print("  'l' - Toggle landmarks")
        print("  'a' - Toggle age/gender detection")
        print("  'q' - Quit")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # AI-powered face detection
            frame, faces = self.ai_detector.detect_faces_ai(frame)
            
            # Get facial landmarks for AR filters
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.ar_filters.face_mesh.process(rgb_frame)
            
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Convert landmarks to pixel coordinates
                    h, w, _ = frame.shape
                    landmarks = []
                    for landmark in face_landmarks.landmark:
                        x = int(landmark.x * w)
                        y = int(landmark.y * h)
                        landmarks.append((x, y))
                    
                    # Apply AR filters
                    if self.current_filter == "sunglasses":
                        self.ar_filters.apply_sunglasses_filter(frame, landmarks)
                    
                    # Show landmarks if enabled
                    if self.show_landmarks:
                        for point in landmarks[::5]:  # Show every 5th landmark
                            cv2.circle(frame, point, 1, (0, 255, 0), -1)
            
            # Show age and gender if enabled
            if self.show_age_gender and faces:
                for face in faces:
                    x, y, w, h = face['bbox']
                    face_roi = frame[y:y+h, x:x+w]
                    if face_roi.size > 0:
                        age, gender = self.age_gender.predict_age_gender(face_roi)
                        cv2.putText(frame, f"{gender}, {age}", (x, y-30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            
            # Display controls on screen
            cv2.putText(frame, f"Filter: {self.current_filter}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            cv2.imshow('Advanced AI Face Detection with AR', frame)
            
            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                self.current_filter = "sunglasses" if self.current_filter != "sunglasses" else "none"
            elif key == ord('l'):
                self.show_landmarks = not self.show_landmarks
            elif key == ord('a'):
                self.show_age_gender = not self.show_age_gender
        
        cap.release()
        cv2.destroyAllWindows()

# ==================================================
# USAGE EXAMPLE
# ==================================================

if __name__ == "__main__":
    # Run the advanced application
    app = AdvancedFaceApp()
    app.run_advanced_detection()

# ==================================================
# ADDITIONAL FEATURES YOU CAN ADD:
# ==================================================

"""
🎯 MORE ADVANCED FEATURES TO IMPLEMENT:

1. REAL-TIME EMOTION RECOGNITION:
  - Use TensorFlow/PyTorch emotion models
  - Detect: Happy, Sad, Angry, Surprised, Fearful, Disgusted, Neutral

2. FACE RECOGNITION & IDENTIFICATION:
  - Use face_recognition library
  - Save known faces to database
  - Real-time identification with names

3. ADVANCED AR FILTERS:
  - Load custom 3D models
  - Animal face morphing
  - Background replacement (green screen effect)

4. GESTURE-CONTROLLED INTERFACE:
  - Hand tracking with MediaPipe
  - Control filters with hand gestures
  - Voice commands integration

5. SOCIAL MEDIA INTEGRATION:
  - Save filtered photos/videos
  - Share to Instagram/TikTok
  - Live streaming with filters

6. PERFORMANCE ANALYTICS:
  - Face detection accuracy metrics
  - FPS optimization
  - Memory usage monitoring

7. MULTI-PERSON FEATURES:
  - Individual filters per person
  - Face swapping between people
  - Group photo enhancements

To implement these, install additional packages:
pip install face-recognition tensorflow torch ultralytics
pip install streamlit (for web interface)
pip install opencv-contrib-python (for advanced features)
"""

✅ AR Filter assets loaded!
⚠️ Age/Gender models not found. Download from OpenCV model zoo.
🚀 Advanced AI Face Detection Started!
Controls:
  's' - Toggle sunglasses filter
  'l' - Toggle landmarks
  'a' - Toggle age/gender detection
  'q' - Quit


'\n🎯 MORE ADVANCED FEATURES TO IMPLEMENT:\n\n1. REAL-TIME EMOTION RECOGNITION:\n  - Use TensorFlow/PyTorch emotion models\n  - Detect: Happy, Sad, Angry, Surprised, Fearful, Disgusted, Neutral\n\n2. FACE RECOGNITION & IDENTIFICATION:\n  - Use face_recognition library\n  - Save known faces to database\n  - Real-time identification with names\n\n3. ADVANCED AR FILTERS:\n  - Load custom 3D models\n  - Animal face morphing\n  - Background replacement (green screen effect)\n\n4. GESTURE-CONTROLLED INTERFACE:\n  - Hand tracking with MediaPipe\n  - Control filters with hand gestures\n  - Voice commands integration\n\n5. SOCIAL MEDIA INTEGRATION:\n  - Save filtered photos/videos\n  - Share to Instagram/TikTok\n  - Live streaming with filters\n\n6. PERFORMANCE ANALYTICS:\n  - Face detection accuracy metrics\n  - FPS optimization\n  - Memory usage monitoring\n\n7. MULTI-PERSON FEATURES:\n  - Individual filters per person\n  - Face swapping between people\n  - Group photo enhancements\n\nTo imple

In [6]:
# AGE/GENDER MODEL SETUP - COMPLETE SOLUTION
# This will automatically download and set up the required models

import cv2
import numpy as np
import os
import urllib.request
import zipfile

class AgeGenderModelDownloader:
    def __init__(self):
        self.model_urls = {
            # Age detection model files
            'age_weights': 'https://github.com/opencv/opencv_extra/raw/master/testdata/dnn/age_net.caffemodel',
            'age_config': 'https://github.com/opencv/opencv_extra/raw/master/testdata/dnn/age_deploy.prototxt',
            
            # Gender detection model files  
            'gender_weights': 'https://github.com/opencv/opencv_extra/raw/master/testdata/dnn/gender_net.caffemodel',
            'gender_config': 'https://github.com/opencv/opencv_extra/raw/master/testdata/dnn/gender_deploy.prototxt'
        }
        
        self.model_files = {
            'age_weights': 'age_net.caffemodel',
            'age_config': 'age_deploy.prototxt', 
            'gender_weights': 'gender_net.caffemodel',
            'gender_config': 'gender_deploy.prototxt'
        }
        
        # Create models directory
        os.makedirs('models', exist_ok=True)

    def download_model_file(self, url, filename):
        """Download a single model file"""
        filepath = os.path.join('models', filename)
        
        if os.path.exists(filepath):
            print(f"✅ {filename} already exists")
            return True
            
        try:
            print(f"⬇️ Downloading {filename}...")
            urllib.request.urlretrieve(url, filepath)
            print(f"✅ Downloaded {filename} successfully")
            return True
        except Exception as e:
            print(f"❌ Error downloading {filename}: {e}")
            return False

    def download_all_models(self):
        """Download all required model files"""
        print("🤖 Downloading Age/Gender Detection Models...")
        print("This may take a few minutes depending on your internet speed.")
        print("-" * 50)
        
        success_count = 0
        for key, url in self.model_urls.items():
            filename = self.model_files[key]
            if self.download_model_file(url, filename):
                success_count += 1
        
        print("-" * 50)
        if success_count == len(self.model_urls):
            print("🎉 All models downloaded successfully!")
            return True
        else:
            print(f"⚠️ Only {success_count}/{len(self.model_urls)} models downloaded")
            return False

# IMPROVED AGE/GENDER DETECTOR WITH AUTO-DOWNLOAD
class ImprovedAgeGenderDetector:
    def __init__(self):
        self.age_net = None
        self.gender_net = None
        self.age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
        self.gender_list = ['Male', 'Female']
        
        # Automatically download and load models
        self.setup_models()
    
    def setup_models(self):
        """Setup models with automatic download"""
        # Check if models exist, download if needed
        model_files = [
            'models/age_net.caffemodel',
            'models/age_deploy.prototxt',
            'models/gender_net.caffemodel', 
            'models/gender_deploy.prototxt'
        ]
        
        models_exist = all(os.path.exists(f) for f in model_files)
        
        if not models_exist:
            print("📦 Age/Gender models not found. Downloading...")
            downloader = AgeGenderModelDownloader()
            if not downloader.download_all_models():
                print("❌ Failed to download models. Using fallback method.")
                self.create_fallback_detector()
                return
        
        try:
            # Load the models
            self.age_net = cv2.dnn.readNet('models/age_net.caffemodel', 'models/age_deploy.prototxt')
            self.gender_net = cv2.dnn.readNet('models/gender_net.caffemodel', 'models/gender_deploy.prototxt')
            print("✅ Age/Gender models loaded successfully!")
            
            # Test the models
            self.test_models()
            
        except Exception as e:
            print(f"❌ Error loading models: {e}")
            self.create_fallback_detector()
    
    def test_models(self):
        """Test if models are working correctly"""
        try:
            # Create a dummy input to test models
            dummy_blob = cv2.dnn.blobFromImage(
                np.zeros((100, 100, 3), dtype=np.uint8), 
                1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746)
            )
            
            # Test age model
            self.age_net.setInput(dummy_blob)
            age_result = self.age_net.forward()
            
            # Test gender model  
            self.gender_net.setInput(dummy_blob)
            gender_result = self.gender_net.forward()
            
            print("✅ Models tested successfully!")
            
        except Exception as e:
            print(f"⚠️ Model test failed: {e}")
    
    def create_fallback_detector(self):
        """Create a simple fallback detector using basic image analysis"""
        print("🔄 Creating fallback age/gender detector...")
        self.use_fallback = True
    
    def predict_age_gender_fallback(self, face_roi):
        """Fallback method using simple image analysis"""
        # Simple heuristic based on face characteristics
        gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        
        # Calculate average brightness (very simple heuristic)
        avg_brightness = np.mean(gray_face)
        
        # Calculate texture (roughness indicator)
        laplacian_var = cv2.Laplacian(gray_face, cv2.CV_64F).var()
        
        # Simple age estimation based on texture
        if laplacian_var > 500:
            age = "(25-32)"  # High texture = adult
        elif laplacian_var > 200:
            age = "(15-20)"  # Medium texture = young adult  
        else:
            age = "(8-12)"   # Low texture = child
        
        # Simple gender estimation (very basic, not accurate)
        gender = "Male" if avg_brightness < 120 else "Female"
        
        return age, gender, 0.5  # Low confidence for fallback
    
    def predict_age_gender(self, face_roi):
        """Predict age and gender for a face region"""
        if face_roi is None or face_roi.size == 0:
            return "Unknown", "Unknown", 0.0
        
        # Use fallback if models not available
        if hasattr(self, 'use_fallback'):
            return self.predict_age_gender_fallback(face_roi)
        
        if self.age_net is None or self.gender_net is None:
            return "Model Error", "Model Error", 0.0
        
        try:
            # Preprocess the face
            blob = cv2.dnn.blobFromImage(
                face_roi, 1.0, (227, 227), 
                (78.4263377603, 87.7689143744, 114.895847746)
            )
            
            # Age prediction
            self.age_net.setInput(blob)
            age_predictions = self.age_net.forward()
            age_idx = age_predictions[0].argmax()
            age_confidence = age_predictions[0].max()
            predicted_age = self.age_list[age_idx]
            
            # Gender prediction
            self.gender_net.setInput(blob)
            gender_predictions = self.gender_net.forward()
            gender_idx = gender_predictions[0].argmax()
            gender_confidence = gender_predictions[0].max()
            predicted_gender = self.gender_list[gender_idx]
            
            # Average confidence
            avg_confidence = (age_confidence + gender_confidence) / 2
            
            return predicted_age, predicted_gender, avg_confidence
            
        except Exception as e:
            print(f"Prediction error: {e}")
            return "Error", "Error", 0.0

# COMPLETE FACE DETECTION WITH AGE/GENDER
class CompleteFaceDetector:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.age_gender_detector = ImprovedAgeGenderDetector()
        
    def detect_with_age_gender(self, frame):
        """Detect faces and predict age/gender"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 5)
        
        results = []
        
        for (x, y, w, h) in faces:
            # Extract face region
            face_roi = frame[y:y+h, x:x+w]
            
            # Predict age and gender
            age, gender, confidence = self.age_gender_detector.predict_age_gender(face_roi)
            
            results.append({
                'bbox': (x, y, w, h),
                'age': age,
                'gender': gender,
                'confidence': confidence
            })
            
            # Draw results on frame
            self.draw_results(frame, x, y, w, h, age, gender, confidence)
        
        return frame, results
    
    def draw_results(self, frame, x, y, w, h, age, gender, confidence):
        """Draw detection results on frame"""
        # Draw face rectangle
        color = (0, 255, 0) if confidence > 0.7 else (0, 255, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        
        # Draw age and gender text
        text = f"{gender}, {age}"
        cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Draw confidence if available
        if confidence > 0:
            conf_text = f"Conf: {confidence:.2f}"
            cv2.putText(frame, conf_text, (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

# DEMO APPLICATION
def run_age_gender_detection():
    """Run the complete age/gender detection demo"""
    detector = CompleteFaceDetector()
    cap = cv2.VideoCapture(0)
    
    print("🎯 Age/Gender Detection Started!")
    print("Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect faces with age/gender
        frame, results = detector.detect_with_age_gender(frame)
        
        # Display statistics
        cv2.putText(frame, f"Faces detected: {len(results)}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Age/Gender Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# MANUAL MODEL DOWNLOAD FUNCTION (Alternative method)
def download_models_manual():
    """Manual model download with progress tracking"""
    print("🔧 Manual Model Download")
    print("If automatic download fails, you can:")
    print("1. Visit: https://github.com/opencv/opencv_extra/tree/master/testdata/dnn")
    print("2. Download these files:")
    print("   - age_net.caffemodel")
    print("   - age_deploy.prototxt") 
    print("   - gender_net.caffemodel")
    print("   - gender_deploy.prototxt")
    print("3. Place them in a 'models' folder")
    print()
    
    # Try automatic download
    downloader = AgeGenderModelDownloader()
    success = downloader.download_all_models()
    
    if success:
        print("✅ Ready to use age/gender detection!")
    else:
        print("❌ Please download models manually using the links above")

# USAGE EXAMPLES
if __name__ == "__main__":
    print("🚀 Age/Gender Detection Setup")
    print("Choose an option:")
    print("1. Download models and run detection")
    print("2. Run with fallback detection (no download needed)")
    
    choice = input("Enter choice (1 or 2): ").strip()
    
    if choice == "1":
        download_models_manual()
        run_age_gender_detection()
    else:
        print("🔄 Using fallback detection...")
        run_age_gender_detection()

# Alternative: Simple age/gender estimation without deep learning models
def simple_age_gender_estimation():
    """
    Simple demonstration without requiring model downloads
    Uses basic image analysis - not very accurate but works immediately
    """
    print("🎯 Simple Age/Gender Estimation (No models required)")
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        
        for (x, y, w, h) in faces:
            face_roi = frame[y:y+h, x:x+w]
            
            # Very simple heuristics (not accurate, just for demo)
            avg_color = np.mean(face_roi)
            face_height_ratio = h / w
            
            # Simple age estimation based on face proportions
            if face_height_ratio > 1.3:
                age_est = "Adult"
            elif face_height_ratio > 1.1:
                age_est = "Teen"
            else:
                age_est = "Child"
            
            # Simple gender estimation (very basic)
            gender_est = "Male" if avg_color < 120 else "Female"
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, f"{gender_est}, {age_est}", (x, y-10), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        
        cv2.imshow('Simple Age/Gender Estimation', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

print("💡 To run simple estimation without downloads: simple_age_gender_estimation()")

🚀 Age/Gender Detection Setup
Choose an option:
1. Download models and run detection
2. Run with fallback detection (no download needed)
🔄 Using fallback detection...
✅ Age/Gender models loaded successfully!
✅ Models tested successfully!
🎯 Age/Gender Detection Started!
Press 'q' to quit
💡 To run simple estimation without downloads: simple_age_gender_estimation()
